### Use SageMaker Feature Store and Apache Spark to generate point-in-time queries to implement Time Travel
The following notebook uses SageMaker Feature Store and Apache Spark to build out a set of Dataframes and queries that provide a pattern for using "Time Travel" capabilities. We will demonstrate how to build a "point-in-time" feature sets by starting with raw transactional data, joining that data with records from the Offline Store, and then building an "entity" dataset to define the items we care about and the timestamp of reference. Techniques include building Spark Dataframes, using outer and inner table joins, using query filters to prune items outside our timeframe, and finally ReduceByKey to reduce the final the dataset. 

#### Install Faker library to help generate timestamps within a given range

In [ ]:
!pip install Faker

In [ ]:
# Faker
from faker import Faker

# Import pyspark and build Spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import max as sql_max
from pyspark.sql.functions import min as sql_min

from pyspark.sql.types import StringType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType

from pyspark import SparkContext, SparkConf
import sagemaker_pyspark
import datetime
import random

# Configure Spark to use the SageMaker Spark dependency jars
classpath = ":".join(sagemaker_pyspark.classpath_jars())


In [ ]:
spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .getOrCreate())


In [ ]:
sc = spark.sparkContext
print(sc.version)

In [ ]:
SEED = 123456
faker = Faker()
faker.seed_locale('en_US', 0)
faker.seed_instance(SEED)

In [ ]:
import sagemaker

BUCKET = sagemaker.Session().default_bucket()
print(BUCKET)

In [ ]:
import os

BASE_PREFIX = "sagemaker-featurestore-blog"

OFFLINE_STORE_BASE_URI = f's3://{BUCKET}/{BASE_PREFIX}'

RAW_PREFIX = os.path.join(BASE_PREFIX, 'raw')
AGG_PREFIX = os.path.join(BASE_PREFIX, 'aggregated')

RAW_FEATURES_PATH_S3 = f"s3://{BUCKET}/{RAW_PREFIX}/"
RAW_FEATURES_PATH_PARQUET = f"s3a://{BUCKET}/{RAW_PREFIX}/"
print(f'S3 Raw Transactions S3 path: {RAW_FEATURES_PATH_S3}')

AGG_FEATURES_PATH_S3 = f"s3://{BUCKET}/{AGG_PREFIX}/"
AGG_FEATURES_PATH_PARQUET = f"s3a://{BUCKET}/{AGG_PREFIX}/"
print(f'S3 Aggregated Data S3 Path: {AGG_FEATURES_PATH_S3}')


In [ ]:
# Feature Store Group requires ISO-8601 string format: yyyy-MM-dd'T'HH:mm:ssZ
# when the EventTime required attribute is type String

ISO_8601_DATETIME_FORMAT = "%Y-%m-%dT%H:%M:%SZ"

In [ ]:
from sagemaker.s3 import S3Downloader

# Find raw transaction data files
raw_file_list = S3Downloader.list(RAW_FEATURES_PATH_S3)
print("Found raw transaction files: \n" + "\n".join(raw_file_list))

In [ ]:
# Find aggregate data files

agg_file_list = S3Downloader.list(AGG_FEATURES_PATH_S3)
print("Found aggregated files: \n" + "\n".join(agg_file_list))

#### Let's retreive our credit card transaction data

In [ ]:
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType, LongType

raw_schema = StructType([StructField('tid', StringType(), True),
                    StructField('event_time', StringType(), True),
                    StructField('cc_num', LongType(), True),
                    StructField('consumer_id', StringType(), True),
                    StructField('amount', DoubleType(), True),
                    StructField('fraud_label', StringType(), True)])

# Build path to transactions data file
raw_file = os.path.join(RAW_FEATURES_PATH_PARQUET, 'transactions.csv')
print(raw_file)

In [ ]:
transactions_df = spark.read.csv(raw_file, header=True, schema=raw_schema)

In [ ]:
transactions_df.printSchema()
transactions_df.count()

In [ ]:
# Show several random rows from dataframe
show_fraction = float(5.0 / 500000.0)
print("Fraction: %2f" % show_fraction)
transactions_sample = transactions_df.sample(withReplacement=False, fraction=show_fraction, seed=3).collect()
transactions_sample

#### Use Sagemaker Client to retrieve info about Feature Group
We will use the `describe_feature_group` method to lookup the S3 Uri location of the Offline Store data files.

In [ ]:
from sagemaker import get_execution_role
import sagemaker
import boto3

role = get_execution_role()
sm_client = boto3.Session().client(service_name='sagemaker')


In [ ]:
# Identify name of the Feature Group that contains aggregated features for our transaction data
FEATURE_GROUP = 'cc-agg-batch-fg'

feature_group_info = sm_client.describe_feature_group(FeatureGroupName=FEATURE_GROUP)
feature_group_info

In [ ]:
# Lookup S3 Location of Offline Store

resolved_offline_store_s3_location = feature_group_info['OfflineStoreConfig']['S3StorageConfig']['ResolvedOutputS3Uri']

# Spark's Parquet file reader requires replacement of 's3' with 's3a'
offline_store_s3a_uri = resolved_offline_store_s3_location.replace("s3:", "s3a:")

print(offline_store_s3a_uri)

In [ ]:
# Read Offline Store data
feature_store_df = spark.read.parquet(offline_store_s3a_uri)

In [ ]:
feature_store_df.printSchema()
feature_store_df.count()

In [ ]:
# Remove records marked as deleted (is_deleted attribute)

feature_store_active_df = feature_store_df.filter(feature_store_df.is_deleted == 'false').drop(feature_store_df.tid)
print(feature_store_active_df.count())

In [ ]:
feature_store_active_df.select('consumer_id', 'avg_amt_last_7d', 'event_time', 'write_time').show(5)

#### Create an enhanced set of features by joining raw transaction data with aggregate features from the Offline Store

In [ ]:
# Join the raw transactons table to the aggregate feature table 

enhanced_df = (transactions_df.join(feature_store_active_df, 
        transactions_df.consumer_id == feature_store_active_df.consumer_id, 
        "left_outer")
    .drop(transactions_df.tid)
    .drop(transactions_df.consumer_id)
    .drop(transactions_df.cc_num)
    .drop(transactions_df.event_time))

In [ ]:
enhanced_df.printSchema()
enhanced_df.count()

In [ ]:
random.sample(enhanced_df.select('consumer_id', 'amount', 'num_trans_last_7d', 'event_time').collect(), 10)

### Building the Time Travel Query

Now that we have an enhanced dataframe with all our transaction data, we can start building the time travel query. We begin be creating an Entity Dataframe which identifies the consumer_ids of interest, coupled with an event_time which represents our cutoff time for that entity. We also define a staleness window which prevents us from using data older than some limit that we define.

### The Entity Dataframe will consist of real Consumer IDs and real event timestamps

First, we need to create an Entity Dataframe consisting of a list or our "target" Consumer IDs, plus a set of realistic timestamps (event_time) to run the point-in-time queries. We also will use a unique (distinct) set of Consumer IDs so that when we run `reduceByKey` later, we will retain a record for each Consumer ID.

In [ ]:
# Num samples in entity dataframe
NUM_RANDOM_SAMPLES = 1000   # was 500

# Create set of tuples with consumer_id and event_time 
cid_ts_tuples = feature_store_active_df.rdd.map(lambda r: (r.consumer_id, r.event_time)).collect()

In [ ]:
cid_samples = random.sample(cid_ts_tuples, NUM_RANDOM_SAMPLES) 
print(len(cid_samples))
cid_samples[0:9]

In [ ]:
# Use sample tuples to build Entity Dataframe containing consumer IDs with actual timestamps

start = datetime.datetime.strptime('2021-03-17T00:00:00Z', ISO_8601_DATETIME_FORMAT)
end = datetime.datetime.strptime('2021-03-31T23:59:59Z', ISO_8601_DATETIME_FORMAT)

samples = list()
for r in range(NUM_RANDOM_SAMPLES):
    row = []
    row.append(cid_samples[r][0])  # consumer_id
    ##row.append(cid_samples[r][1])  # event_time

    # Generate fake timestamp within the above data range
    fake_timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime(ISO_8601_DATETIME_FORMAT)
    row.append(fake_timestamp)
    
    samples.append(row)

print('Count of samples: ' + str(len(samples)))
samples[0:9]

In [ ]:
# Create the actual Entity Dataframe
# (e.g. the dataframe that defines our set of Consumer IDs and timestamps for our point-in-time queries)

entity_df_schema = StructType([
    StructField('consumer_id', StringType(), False),
    StructField('joindate', StringType(), False)
])

In [ ]:
# Create entity data frame

entity_df = spark.createDataFrame(samples, entity_df_schema)
entity_df.show(10)

In [ ]:
# Performance Improvement: 
# This first dataframe filter serves as a performance optimization to reduce the size of dataset
# We compute the overall min and max times for the initial filtering, in one pass

# entity_df used to define bounded time window
minmax_time = entity_df.agg(sql_min("joindate"), sql_max("joindate")).collect()
print(minmax_time)

In [ ]:
min_time, max_time = minmax_time[0]["min(joindate)"], minmax_time[0]["max(joindate)"]
print(f'min_time: {min_time}')
print(f'max_time: {max_time}')

In [ ]:
### OPTIONAL 

# NOTE: This filter is simply a performance optimization
# Filter out records from after query max_time and before staleness window prior to the min_time
# doing this prior to individual {consumer_id, joindate} filtering will speed up subsequent filters

# Choose a "staleness" window of time before which we want to ignore records
allowed_staleness_days = 14

# Eliminate Credit Cards (entities) who do NOT have any relevant records within our time window 
# this window represents the {max_time - min_time} delta, plus our staleness window

# Via the staleness check, we are actually removing items when event_time is MORE than N days before min_time
# Usage: datediff ( enddate, startdate ) - returns days

filtered = enhanced_df.filter(
    (enhanced_df.event_time <= max_time) & 
    (datediff(lit(min_time), enhanced_df.event_time) <= allowed_staleness_days)
)

In [ ]:
filtered.printSchema()
print("After filter, count: " + str(filtered.count()))

In [ ]:
random.sample(filtered.select('consumer_id', 'amount', 'event_time').collect(), 10)

In [ ]:
# Join filtered dataframe with generated entity dataframe; drop duplicate consumer_id field

joined = filtered.join(entity_df, filtered.consumer_id == entity_df.consumer_id, "inner")
joined_df = joined.drop(entity_df.consumer_id)

print("Joined count: " + str(joined_df.count()))

In [ ]:
random.sample(joined_df.select('consumer_id', 'amount', 'event_time').collect(), 10)

In [ ]:
# Filter out data from after query time or before query time minus staleness window
# this query removes events outside the time window for the SPECIFIC Consumer (customer)

allowed_staleness_days = 7

drop_future_and_stale = joined_df.filter(
    (joined_df.event_time <= entity_df.joindate) &
    (datediff(entity_df.joindate, joined_df.event_time) <= allowed_staleness_days)
)

print("After drop stale, count: " + str(drop_future_and_stale.count()))

In [ ]:
random.sample(drop_future_and_stale.select('consumer_id', 'amount', 'event_time').collect(), 10)

In [ ]:
# Use reduceByKey to group by consumer_id and keep most recent record
take_latest = (
    drop_future_and_stale.rdd.map(lambda x: (x.consumer_id, x)) 
    .reduceByKey(
        lambda x, y: x if ((x.event_time) >= (y.event_time)) else y
    )  #  We could have used api_invocation_time as tie-breaker
    .values()  # drop keys
)


In [ ]:
# Convert to DataFrame
latest_df = take_latest.toDF(drop_future_and_stale.schema)

In [ ]:
# Drop extra columns
columns_to_drop = ["joindate", "write_time", "is_deleted", "year", "month", "day", "hour", "query_time", "api_invocation_time"]
final_df = latest_df.drop(*columns_to_drop)

print('Final count: ' + str(final_df.count()))

In [ ]:
# Show final query results

final_df.show(10)

# To save query result to s3:
# OUTPUT_PATH = f"s3://{BUCKET}/{PREFIX}/test_query_output"
# final_df.write.parquet(OUTPUT_PATH, mode="overwrite")